In [1]:
# -*- coding:utf-8 -*-
%pylab inline
from elasticsearch import Elasticsearch
from pprint import pprint as pp
from lshash import LSHash
import base64
import cv2
import numpy as np
print 'OpenCV Version: ' + cv2.__version__

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

Populating the interactive namespace from numpy and matplotlib
OpenCV Version: 3.1.0-dev


# Create an Index

인덱스를 새로 만들어줍니다. 이때 Mapping은 아래의 코드와 같이 만들어줍니다.<br>
[/images/_mapping/samples/](http://localhost:9200/images/_mapping/samples/) 에 들어가면 확인할수 있습니다.

In [78]:
if es.indices.exists(index='images'):
    es.indices.delete(index='images')

In [79]:
mapping = {
    'mappings': {
        'test': {
            "properties": {
                "name": {
                    "type": "string"
                },
                "image": {
                    "type": "image",
                    "feature": {
                        "CEDD": {
                            "hash": [
                                "BIT_SAMPLING",
                                "LSH"
                            ]
                        },
                        "JCD": {
                            "hash": [
                                "BIT_SAMPLING",
                                "LSH"
                            ]
                        },
                        "FCTH": {}
                    },
                "metadata": {
                    "jpeg.image_width": {
                        "type": "string",
                        "store": "yes"
                    },
                    "jpeg.image_height": {
                        "type": "string",
                        "store": "yes"
                    }
                }
                }                
            }
        }
    }
}

es.indices.create(index='images', ignore=400, body=mapping)

{u'acknowledged': True}

# Index Images to Elasticsearch

In [80]:
img = open('../data/images/rainbowsix.jpg', 'rb').read()

body = {
    'name': 'rainboxsix.jpg',
    'image': base64.b64encode(img)
}
# es.create(index='images', doc_type='test', body=body)
es.index(index='images', doc_type='test', id=1, body=body)

{u'_id': u'1',
 u'_index': u'images',
 u'_type': u'test',
 u'_version': 1,
 u'created': True}

# Searching

In [88]:
img2 = open('../data/images/rainbosix_cropped.jpg', 'rb').read()

es.search(index='images', body ={
        'fields': ['name'],
        'query': {
            'image': {
                'image': {
                    'feature': 'JCD',
                    'image': base64.b64encode(img2),
                    'hash': 'LSH',
                    'limit': 10
                }
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'images',
    u'_score': 0.08223946,
    u'_type': u'test',
    u'fields': {u'name': [u'rainboxsix.jpg']}}],
  u'max_score': 0.08223946,
  u'total': 1},
 u'timed_out': False,
 u'took': 93}